In [1]:
import tensorflow as tf
import cv2
import os

2023-06-30 12:45:42.308889: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-30 12:45:42.360287: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-30 12:45:43.135911: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
directory="/var/trainingData/"
print(len(os.listdir(directory)))

2001


In [6]:
data = tf.keras.utils.image_dataset_from_directory(directory)

Found 1177313 files belonging to 2001 classes.


2023-06-30 12:48:56.665656: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-30 12:48:56.669134: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-30 12:48:56.672563: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [ ]:
# Import the LabelEncoder class
from tensorflow.keras.preprocessing import LabelEncoder

# Create an instance of the class
label_encoder = LabelEncoder()

# Fit the encoder to your labels
label_encoder.fit(data.labels)

# Transform your labels into encoded integers
encoded_labels = label_encoder.transform(data.labels)

# Replace the original labels with the encoded labels in your dataset
data = data.map(lambda x, y: (x, encoded_labels))


In [11]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

In [12]:
print(train_size, val_size, test_size)

25754 7358 3679


In [13]:
print(len(data))

36792


In [14]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [15]:
train

<_TakeDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [17]:
# Import the necessary modules
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define the number of classes
num_classes = 2001

# Create a Sequential model
model = keras.Sequential()

# Add the first convolutional layer with 32 filters, 3x3 kernel size, ReLU activation, and input shape of (None, None, 3)
model.add(layers.Conv2D(32, (3, 3), activation="relu", input_shape=(256, 256, 3)))

# Add a max pooling layer with 2x2 pool size
model.add(layers.MaxPooling2D((2, 2)))

# Add a second convolutional layer with 64 filters and ReLU activation
model.add(layers.Conv2D(64, (3, 3), activation="relu"))

# Add another max pooling layer
model.add(layers.MaxPooling2D((2, 2)))

# Add a third convolutional layer with 128 filters and ReLU activation
model.add(layers.Conv2D(128, (3, 3), activation="relu"))

# Add a flatten layer to convert the 3D feature maps to 1D feature vectors
model.add(layers.Flatten())

# Add a dense layer with 256 units and ReLU activation
model.add(layers.Dense(256, activation="relu"))

# Add a dropout layer with 0.5 rate to reduce overfitting
model.add(layers.Dropout(0.5))

# Add the output layer with num_classes units and softmax activation for multi-class classification
model.add(layers.Dense(num_classes, activation="softmax"))

# Print the model summary
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 127, 127, 32)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 60, 60, 128)       73856     
                                                                 
 flatten_1 (Flatten)         (None, 460800)           

In [ ]:
import visualkeras

In [ ]:
visualkeras.layered_view(model).show() # display using your system viewer


In [ ]:
model.summary()

In [ ]:


logdir='logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
